In [3]:
pip install keras-tuner

     |████████████████████████████████| 71kB 9.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=38b946b64c063f6dd4aec9afbceb2d262d7e7e0c246e8c3584ac59583b5e77ec
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=995b500fcbb1a4496b3bc00b7d039799eef3c3a99a6154cffe526a9ca129b9b5
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [4]:
from tensorflow.keras.datasets import cifar10

# Load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Pre-processing
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.


170500096/170498071 [==============================] - 2s 0us/step


In [8]:
# Baseline Model definition

from tensorflow import keras
from tensorflow.keras.layers import (
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    MaxPooling2D
)

INPUT_SHAPE = (32, 32, 3)
NUM_CLASSES = 10
N_EPOCH_SEARCH = 40

model = keras.Sequential()
model.add(
    Conv2D(
        filters=16,
        kernel_size=3,
        activation='relu',
        input_shape=INPUT_SHAPE
    )
)
model.add(Conv2D(16, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(rate=0.25))
model.add(Conv2D(32, 3, activation='relu'))
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [7]:
model.fit(x_train, y_train, epochs=N_EPOCH_SEARCH, validation_split=0.1)

Epoch 1/40
1407/1407 [==============================] - 38s 5ms/step - loss: 1.6078 - accuracy: 0.4086 - val_loss: 1.2999 - val_accuracy: 0.5418
Epoch 2/40
1407/1407 [==============================] - 6s 4ms/step - loss: 1.2713 - accuracy: 0.5432 - val_loss: 1.0963 - val_accuracy: 0.6156
Epoch 3/40
1407/1407 [==============================] - 6s 4ms/step - loss: 1.1389 - accuracy: 0.5954 - val_loss: 1.0417 - val_accuracy: 0.6256
Epoch 4/40
1407/1407 [==============================] - 6s 4ms/step - loss: 1.0528 - accuracy: 0.6276 - val_loss: 0.9757 - val_accuracy: 0.6512
Epoch 5/40
1407/1407 [==============================] - 6s 4ms/step - loss: 0.9946 - accuracy: 0.6453 - val_loss: 0.9366 - val_accuracy: 0.6666
Epoch 6/40
1407/1407 [==============================] - 6s 4ms/step - loss: 0.9467 - accuracy: 0.6654 - val_loss: 0.8583 - val_accuracy: 0.7010
Epoch 7/40
1407/1407 [==============================] - 6s 4ms/step - loss: 0.9097 - accuracy: 0.6794 - val_loss: 0.7881 - val_accuracy

In [12]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"loss: {loss}, accuracy: {accuracy}")

313/313 [==============================] - 1s 3ms/step - loss: 2.3046 - accuracy: 0.1039
loss: 2.304551601409912, accuracy: 0.1039000004529953


In [14]:
# Keras Tuner Hypermodels

from kerastuner import HyperModel


class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = keras.Sequential()
        model.add(
            Conv2D(
                filters=16,
                kernel_size=3,
                activation='relu',
                input_shape=self.input_shape
            )
        )
        model.add(
            Conv2D(
                filters=16,
                activation='relu',
                kernel_size=3
            )
        )
        model.add(MaxPooling2D(pool_size=2))
        model.add(
            Dropout(rate=hp.Float(
                'dropout_1',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            ))
        )
        model.add(
            Conv2D(
                filters=32,
                kernel_size=3,
                activation='relu'
            )
        )
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'num_filters',
                    values=[32, 64],
                    default=64,
                ),
                activation='relu',
                kernel_size=3
            )
        )
        model.add(MaxPooling2D(pool_size=2))
        model.add(
            Dropout(rate=hp.Float(
                'dropout_2',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            ))
        )
        model.add(Flatten())
        model.add(
            Dense(
                units=hp.Int(
                    'units',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=128
                ),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'
                )
            )
        )
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_3',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

In [19]:
# Random Search HP Tuning
from kerastuner.tuners import RandomSearch

NUM_CLASSES = 10  # cifar10 number of classes
INPUT_SHAPE = (32, 32, 3)  # cifar10 images input shape
SEED = 1
MAX_TRIALS = 20
EXECUTION_PER_TRIAL = 2

hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

tuner_rs = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    seed=SEED,
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='random_search',
    project_name='cifar10'
)

In [20]:
# Hyperband
from kerastuner.tuners import Hyperband

HYPERBAND_MAX_EPOCHS = 40

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_accuracy',
    seed=SEED,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='hyperband',
    project_name='cifar10'
)


INFO:tensorflow:Reloading Oracle from existing project hyperband/cifar10/oracle.json
INFO:tensorflow:Reloading Tuner from hyperband/cifar10/tuner0.json


In [21]:
tuner_rs.search_space_summary()

Search space summary
Default search space size: 7
dropout_1 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
num_filters (Choice)
{'default': 64, 'conditions': [], 'values': [32, 64], 'ordered': True}
dropout_2 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
units (Int)
{'default': 128, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
dense_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
dropout_3 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
learning_rate (Float)
{'default': 0.001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [ ]:
N_EPOCH_SEARCH = 40

tuner_rs.search(x_train, y_train, epochs=N_EPOCH_SEARCH, validation_split=0.1)

Trial 19 Complete [00h 08m 37s]
val_accuracy: 0.7618999779224396

Best val_accuracy So Far: 0.7947999835014343
Total elapsed time: 02h 42m 36s

Search: Running Trial #20

Hyperparameter    |Value             |Best Value So Far 
dropout_1         |0.25              |0.2               
num_filters       |64                |64                
dropout_2         |0.3               |0.25              
units             |64                |384               
dense_activation  |relu              |tanh              
dropout_3         |0                 |0.1               
learning_rate     |0.0034929         |0.00051617        

Epoch 1/40
1407/1407 [==============================] - 7s 5ms/step - loss: 1.6829 - accuracy: 0.3763 - val_loss: 1.5379 - val_accuracy: 0.4556
Epoch 2/40
1407/1407 [==============================] - 6s 5ms/step - loss: 1.3842 - accuracy: 0.4996 - val_loss: 1.2589 - val_accuracy: 0.5526
Epoch 3/40
1407/1407 [==============================] - 7s 5ms/step - loss: 1.2743 -

In [ ]:
# Show a summary of the search
tuner.results_summary()

# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model.
loss, accuracy = best_model.evaluate(x_test, y_test)